In [1]:
import pyprind
import pandas as pd
import os

"""basepath = 'aclImdb'

labels = {'pos' : 1, 'neg' : 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()"""

"basepath = 'aclImdb'\n\nlabels = {'pos' : 1, 'neg' : 0}\npbar = pyprind.ProgBar(50000)\ndf = pd.DataFrame()"

In [2]:
"""for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()

df.columns = ['review', 'sentiment']"""

"for s in ('test', 'train'):\n    for l in ('pos', 'neg'):\n        path = os.path.join(basepath, s, l)\n        for file in sorted(os.listdir(path)):\n            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:\n                txt = infile.read()\n            df = df.append([[txt, labels[l]]], ignore_index=True)\n            pbar.update()\n\ndf.columns = ['review', 'sentiment']"

In [3]:
import numpy as np
np.random.seed(0)
"""df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')"""

"df = df.reindex(np.random.permutation(df.index))\ndf.to_csv('movie_data.csv', index=False, encoding='utf-8')"

In [4]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.shape

(50000, 2)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()#создаем 1-граммы, чтобы создать 2-граммы передаем ngram_range = (2, 2)
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)
count.vocabulary_#глосарий - слово и индекс

{'the': 6,
 'sun': 4,
 'is': 1,
 'shining': 3,
 'weather': 8,
 'sweet': 5,
 'and': 0,
 'one': 2,
 'two': 7}

In [6]:
count = CountVectorizer(ngram_range = (2, 2))
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)
count.vocabulary_

{'the sun': 9,
 'sun is': 7,
 'is shining': 1,
 'the weather': 10,
 'weather is': 11,
 'is sweet': 2,
 'shining the': 6,
 'sweet and': 8,
 'and one': 0,
 'one and': 4,
 'one is': 5,
 'is two': 3}

In [7]:
bag.toarray()#вектора признаков, сырые частоты термов 
#tf(t, d) - сколько раз терм t встречается в документе d

array([[0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int64)

In [8]:
#мера частота терма - обратная частота документа позволяет найти часто встречающиеся слова
#которые не имеют смысла tf-idf (t, d) = tf(t, d) * idf(t, d)
#idf(t, d) = log(nd/(1 + df(t, d))) nd - общее количество документов
#df(t, d) - количество документов d содержащих терм t

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',#нормализуем вектор термов деля на L2 норму
                         smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.58 0.   0.   0.   0.   0.   0.58 0.   0.58 0.   0.  ]
 [0.   0.   0.58 0.   0.   0.   0.   0.   0.   0.   0.58 0.58]
 [0.57 0.22 0.22 0.28 0.28 0.28 0.28 0.22 0.28 0.22 0.22 0.22]]


In [10]:
#в sklearn формулы отличаются, а именно:
#idf(t, d) = log(1 + nd/(1 + df(t, d)))
#tf-idf (t, d) = tf(t, d) * (idf(t, d) + 1) из-за того что smooth_idf=True чтобы  = 0 для термов встречающихся во всех документах


In [11]:
df.loc[0, 'review'][-50:] #данные необходимо очитстить

'is seven.<br /><br />Title (Brazil): Not Available'

In [12]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) #удаление html разметки
    emoticons = re.findall('(?::|;|=)(?:-)?(?:-\)|\(|D|Р)',
                           text)#находми все эмотиконы
    text = (re.sub('[\W]+', ' ', text.lower()) + #находим и удалям все несловарные символы
            ' '.join(emoticons).replace('-', ''))#возвращаем эмотиконы
    return text
    

In [13]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [14]:
df['review'] = df['review'].apply(preprocessor)

In [15]:
def tokenizer(text):#один из вариантов разбиения на лексемы
    return(text.split())
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [16]:
#аналог - стемминг - преобразование слова в его корневую форму
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [17]:
#удаление стоп-слов - слова, которые распространены в тексте и вероятно не несут в себе полезной информации
import nltk
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('runners like running and thus they run')[-10:]
 if w not in stop]

['runner', 'like', 'run', 'thu', 'run']

In [18]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [19]:
"""from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer,
                                  tokenizer_porter],
               'clf__penalty': ['ll', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer,
                                    tokenizer_porter],
               'vect__use_idf': [False],
               'vect__norm' : [None],
               'clf__penalty': ['11', '12'],
               'clf__C': [1.0, 10.0, 100.0]}]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf',
                      LogisticRegression(random_state=0,
                                         solver='liblinear'))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5, verbose=2,
                           n_jobs=-1)
gs_lr_tfidf.fit(X_train, y_train)"""

"from sklearn.model_selection import GridSearchCV\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.feature_extraction.text import TfidfVectorizer\n\ntfidf = TfidfVectorizer(strip_accents=None,\n                        lowercase=False,\n                        preprocessor=None)\n\nparam_grid = [{'vect__ngram_range': [(1,1)],\n               'vect__stop_words': [stop, None],\n               'vect__tokenizer': [tokenizer,\n                                  tokenizer_porter],\n               'clf__penalty': ['ll', 'l2'],\n               'clf__C': [1.0, 10.0, 100.0]},\n              {'vect__ngram_range': [(1,1)],\n               'vect__stop_words': [stop, None],\n               'vect__tokenizer': [tokenizer,\n                                    tokenizer_porter],\n               'vect__use_idf': [False],\n               'vect__norm' : [None],\n               'clf__penalty': ['11', '12'],\n               'clf__C': [1.0, 10.0, 100.0]}]

In [20]:
#динамическое обучение
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text) #удаление html разметки
    emoticons = re.findall('(?::|;|=)(?:-)?(?:-\)|\(|D|Р)',
                           text)#находми все эмотиконы
    text = (re.sub('[\W]+', ' ', text.lower()) + #находим и удалям все несловарные символы
            ' '.join(emoticons).replace('-', ''))#возвращаем эмотиконы
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized
    

In [21]:
def stream_docs(path):
    with open(path, 'r', encoding = 'utf-8') as csv:
        next(csv)
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [22]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [23]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,#большое кол-во признаков - меньше шанс коллизии 
                         preprocessor=None,
                         tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1)
doc_stream = stream_docs(path='movie_data.csv')

In [24]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
#обучаем на 45000 данных из набора
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

c:\Users\VIKTOR\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:27


In [25]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
#проверяем на следующих 5000
X_test = vect.transform(X_test)
print(clf.score(X_test, y_test))

0.8682


In [26]:
clf = clf.partial_fit(X_test, y_test)#дообучаем на тех данных на которых проверяли

In [27]:
#современная модель суммирования слов - word2vec

In [28]:
#Тематическое моделирование - тема для непоименнованного документа
#латентное размещение Дирихле (LDA) - отыскивает группы слов часто появляющихся вместе
#раскладывает матрицу суммирования слов на матрицу отображения документов на темы
#и матрицу отображения слов на темы при перемножении они дадут матрицу суммирования слов
#результат - темы в матрице суммирования слов

In [29]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
count = CountVectorizer(stop_words='english',
                        max_df=.1,#максимальная частота слов подлежащих рассмотрению избежать (is, and, a, the)
                        max_features=5000)#ограничение на размерность данных
X = count.fit_transform(df['review'].values)

In [30]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch',#за 1 раз все, аналог online (динамическое обучение)
                                n_jobs=-1)
X_topics = lda.fit_transform(X)

In [31]:
lda.components_.shape #матрица значения важности слов (5000) для каждой из 10 тем

(10, 5000)

In [32]:
#выведем 5 самых важных слов для каждой темы (в матрице в порядке возрастания)
n_top_words = 5
feature_names = count.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print(f'Тема №{topic_idx + 1}')
    print(' '.join([feature_names[i]
                    for i in topic.argsort()
                        [:-n_top_words - 1: -1]]))

Тема №1
worst minutes awful script stupid
Тема №2
family mother father children kids
Тема №3
american war dvd music black
Тема №4
human audience art cinema feel
Тема №5
police guy car dead murder
Тема №6
horror house sex blood woman
Тема №7
role performance comedy actor performances
Тема №8
series episode episodes tv season
Тема №9
book version original effects fi
Тема №10
action fight guy guys cool


In [33]:
#в целом плохие фильмы
#Семейные фильмы
#Военные фильмы
#фильмы об искусстве
#криминальные фильмы
#фильмы ужасов
#комедийные фильмы
#фильмы связанные с ТВ шоу
#по мотивам книг
#фильмы боевики

In [34]:
horror = X_topics[:, 5].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
    print(f'\nФильм ужасов №{iter_idx + 1}')
    print(df['review'][movie_idx][:300], '...')


Фильм ужасов №1
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...

Фильм ужасов №2
Before I talk about the ending of this film I will talk about the plot. Some dude named Gerald breaks his engagement to Kitty and runs off to Craven Castle in Scotland. After several months Kitty and her aunt venture off to Scottland. Arriving at Craven Castle Kitty finds that Gerald has aged and he ...

Фильм ужасов №3
This film marked the end of the "serious" Universal Monsters era (Abbott and Costello meet up with the monsters later in "Abbott and Costello Meet Frankentstein"). It was a somewhat desparate, yet fun attempt to revive the classic monsters of the Wolf Man, Frankenstein's monster, and Dracula one "la ...


In [35]:
#сохраняем последнюю модель
import pickle
dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)
pickle.dump(stop,
            open(os.path.join(dest, 'stopwords.pkl'), 'wb'),#запись в двоичном режиме
            protocol=4)
pickle.dump(clf,
            open(os.path.join(dest, 'classifier.pkl'), 'wb'),
            protocol=4)